In [16]:
from langchain_community.document_loaders import WebBaseLoader
from pprint import pprint
# List of URLs you want to load (We will crawl the entire site later)
urls = [
    "https://docs.nvidia.com",
    "https://docs.nvidia.com/cuda",
    "https://docs.nvidia.com/deeplearning",
    "https://docs.nvidia.com/gameworks"
    "https://docs.nvidia.com/cudnn",
    "https://docs.nvidia.com/tensorrt",
    "https://rosecrisp.com",
    
    
]

data = []

# Loop through each URL and load the page content
for url in urls:
    loader = WebBaseLoader(url)
    page = loader.load()
    page[0].page_content = page[0].page_content.replace('\n', '')
    data.extend(page)
    

pprint(data)


[Document(metadata={'source': 'https://docs.nvidia.com', 'title': 'NVIDIA Documentation Hub - NVIDIA Docs', 'description': 'Get started by exploring the latest technical information and product documentation', 'language': 'en'}, page_content="NVIDIA Documentation Hub - NVIDIA DocsSubmit SearchNVIDIA DeveloperBlogForumsJoinSubmit SearchNVIDIA DeveloperBlogForumsJoinMenuNVIDIA Documentation HubGet started by exploring the latest technical information and product documentationBrowse byFeaturedProductsAll DocumentsSubmit SearchMost PopularGet support for our latest innovations and see how you can bring them into your own work.NVIDIA API Documentation        Your guide to NVIDIA APIs including NIM and CUDA-X microservices.                Browse            NVIDIA AI Enterprise        NVIDIA AI Enterprise is an end-to-end, cloud-native software platform that accelerates data science pipelines and streamlines development and deployment of production-grade co-pilots and other generative AI appl

In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_docs(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    return text_splitter.split_documents(data)



In [20]:
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")        

In [21]:
# Add to ChromaDB vector store
from langchain_community.vectorstores import Chroma

all_splits = split_docs(data)  # Define the variable "all_splits" here

vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-chroma",
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()
        

In [22]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions



# Setup vector database
client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

# Populate vector database
vectorstore = Weaviate.from_documents(
    client = client,    
    documents = all_splits,
    embedding = embeddings,
    by_text = False
)

# Define vectorstore as retriever to enable semantic search
retriever = vectorstore.as_retriever()        

embedded weaviate is already listening on port 8079


/Users/rosecrisp/anaconda3/lib/python3.11/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.5. The latest version is 4.6.5.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(
{"level":"info","msg":"Created shard langchain_29f4f0f09a484e8a83f7c828cce6c77e_6Rv4zJ1T6RqN in 2.368084ms","time":"2024-07-17T10:31:35-04:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-07-17T10:31:35-04:00","took":46958}


In [23]:
from langchain.llms import Ollama

ollama = Ollama(base_url="http://localhost:11434", model="autopilot")

In [24]:
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question only from the following context:
{context}

Question: {question}

"""
prompt = ChatPromptTemplate.from_template(template)


In [25]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


# Chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | ollama
    | StrOutputParser()
)        

{"action":"restapi_management","level":"info","msg":"Shutting down... ","time":"2024-07-17T10:31:52-04:00"}
{"action":"restapi_management","level":"info","msg":"Stopped serving weaviate at http://127.0.0.1:8079","time":"2024-07-17T10:31:52-04:00"}


In [33]:
# Question
rag_chain.invoke("who is rose crisp")        

'I am unable to provide information about Rose Crisp as I have not been provided with enough context or details. If you have any specific questions related to her, please provide me with more information so that I can assist you accordingly.'

In [29]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms.ollama import Ollama
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Initialize the Ollama model
ollama = Ollama(base_url="http://localhost:11434", model="autopilot")

# Function to handle the input and display the response
def handle_query(sender):
    with output:
        clear_output(wait=True)  # Ensure the output is cleared only once ready to display new output
        print("Processing...")
        try:
            response = rag_chain.invoke(input_box.value)
            display(HTML(f"<div style='word-wrap: break-word; white-space: pre-wrap;'>Response: {response}</div>"))
        except Exception as e:
            print("An error occurred:", str(e))

# Create widgets for input and output
input_box = widgets.Text(description="Enter a query:")
button = widgets.Button(description="Submit Query")
output = widgets.Output()

# Set up the button's event to handle the query
button.on_click(handle_query)

# Display the widgets
display(input_box, button, output)

/Users/rosecrisp/anaconda3/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Text(value='', description='Enter a query:')

Button(description='Submit Query', style=ButtonStyle())

Output()